# Julia *gotchas* and how to handle them
(Inspired by http://www.stochasticlifestyle.com/7-julia-gotchas-handle/ by Chris Rackauckas.)

**One can write terribly slow code in any language, including Julia.**

Below we address common performance *gotchas* in Julia code.

# Gotcha 1: Global scope

In [3]:
a=2.0
b=3.0
function linearcombo()
  return 2a+b
end
answer = linearcombo()

@show answer;

answer = 7.0


In [6]:
@time linearcombo()

  0.000003 seconds (6 allocations: 192 bytes)


7.0

In [7]:
a = 3

3

In [8]:
linearcombo()

9.0

The issue here is that the REPL/global scope does not guarantee that `a` and `b` are of a certain type.

In [2]:
@code_llvm linearcombo()


;  @ In[1]:4 within `linearcombo'
; Function Attrs: uwtable
define nonnull %jl_value_t addrspace(10)* @julia_linearcombo_17537() #0 {
top:
  %0 = alloca %jl_value_t addrspace(10)*, i32 2
  %gcframe = alloca %jl_value_t addrspace(10)*, i32 4
  %1 = bitcast %jl_value_t addrspace(10)** %gcframe to i8*
  call void @llvm.memset.p0i8.i32(i8* %1, i8 0, i32 32, i32 0, i1 false)
  %2 = call %jl_value_t*** inttoptr (i64 1720369472 to %jl_value_t*** ()*)() #6
  %3 = getelementptr %jl_value_t addrspace(10)*, %jl_value_t addrspace(10)** %gcframe, i32 0
  %4 = bitcast %jl_value_t addrspace(10)** %3 to i64*
  store i64 4, i64* %4
  %5 = getelementptr %jl_value_t**, %jl_value_t*** %2, i32 0
  %6 = getelementptr %jl_value_t addrspace(10)*, %jl_value_t addrspace(10)** %gcframe, i32 1
  %7 = bitcast %jl_value_t addrspace(10)** %6 to %jl_value_t***
  %8 = load %jl_value_t**, %jl_value_t*** %5
  store %jl_value_t** %8, %jl_value_t*** %7
  %9 = bitcast %jl_value_t*** %5 to %jl_value_t addrspace(10)***
  st

### How to identify and avoid this issue?

One way to identify the issue is [Traceur.jl](https://github.com/MikeInnes/Traceur.jl). It is basically a codified version of the [performance tips](https://docs.julialang.org/en/v0.6.4/manual/performance-tips/#man-performance-tips-1) in the Julia documentation.

In [9]:
using Traceur
@trace linearcombo()

┌ Warning: uses global variable Main.a
└ @ In[3]:-1
┌ Warning: uses global variable Main.b
└ @ In[3]:-1
┌ Warning: dynamic dispatch to 2 * Main.a
└ @ In[3]:-1
┌ Warning: dynamic dispatch to 2 * Main.a + Main.b
└ @ In[3]:-1
┌ Warning: linearcombo returns Any
└ @ In[3]:4


9.0

#### 1) Wrap code in functions.

In [10]:
function outer()
    a=2.0; b=3.0
    function linearcombo()
      return 2a+b
    end
    return linearcombo() 
end

answer = outer()

@show answer;

answer = 7.0


In [ ]:
# myscript.jl

function myfunc(a,b)
    a+b
end

g(a) = a^2

function main()
    a = 2.0
    b = 4.2
    z = myfunc(a,b)
    println(g(z))
end

main()

In [11]:
@code_llvm outer()


;  @ In[10]:2 within `outer'
; Function Attrs: uwtable
define double @julia_outer_18376() #0 {
top:
  ret double 7.000000e+00
}


This is fast.

In fact, it's not just fast, but as fast as it can be! Julia has figured out the result of the calculation at compile-time and returns **just the result (a literal)!**

(Effectively, `outer() = 7` at run-time.)

In [ ]:
@trace outer()

#### 2) Declare globals as (compile-time) constants.

In [12]:
const A=2.0
const B=3.0

function Linearcombo()
  return 2A+B
end
answer = Linearcombo()

@show answer;

answer = 7.0


In [13]:
@code_llvm Linearcombo()


;  @ In[12]:5 within `Linearcombo'
; Function Attrs: uwtable
define double @julia_Linearcombo_18378() #0 {
top:
  ret double 7.000000e+00
}


In [15]:
@trace Linearcombo()

7.0

Note that the constants above are only compile-time constants, which can be modified:

In [16]:
const A=1.0

1.0

In [17]:
Linearcombo() # still returns 7, not 5

7.0

#### Take home message: Always wrap a performance critical piece of code in a self-contained function.

# Gotcha 2: Type-instabilities

What's bad for performance about the following function?

In [18]:
function g()
  x=1
  for i = 1:10
    x = x/2
  end
  return x
end

g (generic function with 1 method)

In [19]:
@code_llvm debuginfo=:none g()


; Function Attrs: uwtable
define double @julia_g_18500() #0 {
top:
  br label %L2

L2:                                               ; preds = %top, %L29
  %0 = phi double [ 4.940660e-324, %top ], [ %value_phi1, %L29 ]
  %.sroa.012.0 = phi i64 [ 1, %top ], [ %4, %L29 ]
  %tindex_phi = phi i2 [ -2, %top ], [ 1, %L29 ]
  %value_phi = phi i64 [ 1, %top ], [ %3, %L29 ]
  switch i2 %tindex_phi, label %L17 [
    i2 -2, label %L6
    i2 1, label %L19
  ]

L6:                                               ; preds = %L2
  %1 = sitofp i64 %.sroa.012.0 to double
  br label %L19

L17:                                              ; preds = %L2
  call void @jl_throw(%jl_value_t addrspace(12)* addrspacecast (%jl_value_t* inttoptr (i64 111006864 to %jl_value_t*) to %jl_value_t addrspace(12)*))
  unreachable

L19:                                              ; preds = %L2, %L6
  %value_phi1.in = phi double [ %1, %L6 ], [ %0, %L2 ]
  %value_phi1 = fmul double %value_phi1.in, 5.000000e-01
  %2 = icmp eq

A more drastic example

In [20]:
f() = rand([1.0, 2, "3"])

f (generic function with 1 method)

In [21]:
@code_llvm debuginfo=:none f()


; Function Attrs: uwtable
define nonnull %jl_value_t addrspace(10)* @julia_f_18523() #0 {
top:
  %0 = alloca %jl_value_t addrspace(10)*, i32 2
  %gcframe = alloca %jl_value_t addrspace(10)*, i32 4
  %1 = bitcast %jl_value_t addrspace(10)** %gcframe to i8*
  call void @llvm.memset.p0i8.i32(i8* %1, i8 0, i32 32, i32 0, i1 false)
  %2 = alloca { i64, i64, i64, i64 }, align 8
  %3 = call %jl_value_t*** inttoptr (i64 1720369472 to %jl_value_t*** ()*)() #9
  %4 = getelementptr %jl_value_t addrspace(10)*, %jl_value_t addrspace(10)** %gcframe, i32 0
  %5 = bitcast %jl_value_t addrspace(10)** %4 to i64*
  store i64 4, i64* %5
  %6 = getelementptr %jl_value_t**, %jl_value_t*** %3, i32 0
  %7 = getelementptr %jl_value_t addrspace(10)*, %jl_value_t addrspace(10)** %gcframe, i32 1
  %8 = bitcast %jl_value_t addrspace(10)** %7 to %jl_value_t***
  %9 = load %jl_value_t**, %jl_value_t*** %6
  store %jl_value_t** %9, %jl_value_t*** %8
  %10 = bitcast %jl_value_t*** %6 to %jl_value_t addrspace(10)***
 

### How to find and deal with type-instabilities

#### 1) Avoid type changes

Initialize `x` as `Float64` and it's fast.

In [22]:
function h()
  x=1.0
  for i = 1:10
    x = x/2
  end
  return x
end

h (generic function with 1 method)

In [23]:
@code_llvm debuginfo=:none h()


; Function Attrs: uwtable
define double @julia_h_18537() #0 {
top:
  ret double 0x3F50000000000000
}


In [24]:
h()

0.0009765625

#### 2) Detect issues with `@code_warntype` (or `@trace`)

In [25]:
@code_warntype g()

Variables
  #self#::Core.Compiler.Const(g, false)
  x::Union{Float64, Int64}
  @_3::Union{Nothing, Tuple{Int64,Int64}}
  i::Int64

Body::Float64
1 ─       (x = 1)
│   %2  = (1:10)::Core.Compiler.Const(1:10, false)
│         (@_3 = Base.iterate(%2))
│   %4  = (@_3::Core.Compiler.Const((1, 1), false) === nothing)::Core.Compiler.Const(false, false)
│   %5  = Base.not_int(%4)::Core.Compiler.Const(true, false)
└──       goto #4 if not %5
2 ┄ %7  = @_3::Tuple{Int64,Int64}::Tuple{Int64,Int64}
│         (i = Core.getfield(%7, 1))
│   %9  = Core.getfield(%7, 2)::Int64
│         (x = x / 2)
│         (@_3 = Base.iterate(%2, %9))
│   %12 = (@_3 === nothing)::Bool
│   %13 = Base.not_int(%12)::Bool
└──       goto #4 if not %13
3 ─       goto #2
4 ┄       return x::Float64


(On a side note: since the type can only vary between `Float64` and `Int64`, Julia can still produce reasonable code by *union splitting*. See the blog post by Tim Holy: https://julialang.org/blog/2018/08/union-splitting)

In [26]:
@code_warntype h()

Variables
  #self#::Core.Compiler.Const(h, false)
  x::Float64
  @_3::Union{Nothing, Tuple{Int64,Int64}}
  i::Int64

Body::Float64
1 ─       (x = 1.0)
│   %2  = (1:10)::Core.Compiler.Const(1:10, false)
│         (@_3 = Base.iterate(%2))
│   %4  = (@_3::Core.Compiler.Const((1, 1), false) === nothing)::Core.Compiler.Const(false, false)
│   %5  = Base.not_int(%4)::Core.Compiler.Const(true, false)
└──       goto #4 if not %5
2 ┄ %7  = @_3::Tuple{Int64,Int64}::Tuple{Int64,Int64}
│         (i = Core.getfield(%7, 1))
│   %9  = Core.getfield(%7, 2)::Int64
│         (x = x / 2)
│         (@_3 = Base.iterate(%2, %9))
│   %12 = (@_3 === nothing)::Bool
│   %13 = Base.not_int(%12)::Bool
└──       goto #4 if not %13
3 ─       goto #2
4 ┄       return x


#### 3) The C/Fortran way: specify types (to get errors or to heal the problem by conversion)

In [27]:
function g2()
  x::Int64 = 1
  for i = 1:10
    x = x/2
  end
  return x
end

g2 (generic function with 1 method)

In [28]:
g2()

InexactError: InexactError: Int64(0.5)

In [29]:
function g3()
  x::Float64 = 1 # triggers an implicit conversion to Float64
  for i = 1:10
    x = x/2
  end
  return x
end

g3 (generic function with 1 method)

In [30]:
@code_llvm debuginfo=:none g3()


; Function Attrs: uwtable
define double @julia_g3_18675() #0 {
top:
  ret double 0x3F50000000000000
}


#### 4) Function barriers

In [31]:
data = Union{Int64,Float64,String}[4, 2.0, "test", 3.2, 1]

5-element Array{Union{Float64, Int64, String},1}:
 4      
 2.0    
  "test"
 3.2    
 1      

In [32]:
function calc_square(x)
  for i in eachindex(x)
    val = x[i]
    val^2
  end
end

calc_square (generic function with 1 method)

In [33]:
@code_warntype calc_square(data)

Variables
  #self#::Core.Compiler.Const(calc_square, false)
  x::Array{Union{Float64, Int64, String},1}
  @_3::Union{Nothing, Tuple{Int64,Int64}}
  i::Int64
  val::Union{Float64, Int64, String}

Body::Nothing
1 ─ %1  = Main.eachindex(x)::Base.OneTo{Int64}
│         (@_3 = Base.iterate(%1))
│   %3  = (@_3 === nothing)::Bool
│   %4  = Base.not_int(%3)::Bool
└──       goto #4 if not %4
2 ┄ %6  = @_3::Tuple{Int64,Int64}::Tuple{Int64,Int64}
│         (i = Core.getfield(%6, 1))
│   %8  = Core.getfield(%6, 2)::Int64
│         (val = Base.getindex(x, i))
│   %10 = val::Union{Float64, Int64, String}
│   %11 = Core.apply_type(Base.Val, 2)::Core.Compiler.Const(Val{2}, false)
│   %12 = (%11)()::Core.Compiler.Const(Val{2}(), false)
│         Base.literal_pow(Main.:^, %10, %12)
│         (@_3 = Base.iterate(%1, %8))
│   %15 = (@_3 === nothing)::Bool
│   %16 = Base.not_int(%15)::Bool
└──       goto #4 if not %16
3 ─       goto #2
4 ┄       return


In [34]:
function calc_square_outer(x)
  for i in eachindex(x)
    calc_square_inner(x[i])
  end
end

calc_square_inner(x) = x^2

calc_square_inner (generic function with 1 method)

In [37]:
@code_warntype calc_square_inner(data[3])

Variables
  #self#::Core.Compiler.Const(calc_square_inner, false)
  x::String

Body::String
1 ─ %1 = Core.apply_type(Base.Val, 2)::Core.Compiler.Const(Val{2}, false)
│   %2 = (%1)()::Core.Compiler.Const(Val{2}(), false)
│   %3 = Base.literal_pow(Main.:^, x, %2)::String
└──      return %3


#### Comments:

Why allow type-instabilities in the first place? Convenience vs performance tradeoff.

Note that type instabilities can naturally occur (reading files, user input etc.) so not any red marker is bad/avoidable.

Note that Julia is smart and a changing type isn't *per se* an issue:

In [38]:
function g4()
  x=1
  x=1.0
  for i = 1:10
    x = x/2
  end
  return x
end

g4 (generic function with 1 method)

In [39]:
@code_llvm debuginfo=:none g4()


; Function Attrs: uwtable
define double @julia_g4_18769() #0 {
top:
  ret double 0x3F50000000000000
}


#### Take home message: watch out for type-instabilities in performance critical parts of your code.

# Gotcha 3: Views and copies

Say we were facing the following task: Given a 3x3 matrix M and a vector v calculate the dot product between the first column of M and v.

In [41]:
using BenchmarkTools, LinearAlgebra

M = rand(3,3);
x = rand(3);

In [42]:
f(x,M) = dot(M[1:3,1], x)
@btime f($x,$M);

  43.692 ns (1 allocation: 112 bytes)


In [43]:
g(x,M) = dot(view(M, 1:3,1), x)
@btime g($x, $M);

  19.759 ns (1 allocation: 64 bytes)


In [44]:
g(x,M) = @views dot(M[1:3,1], x)
@btime g($x, $M);

  19.559 ns (1 allocation: 64 bytes)


# Gotcha 4: Temporary allocations and vectorized code

In [45]:
using BenchmarkTools

In [46]:
function f()
  x = [1;5;6]
  for i in 1:100_000
    x = x + 2*x
  end
  return x
end

f (generic function with 2 methods)

In [47]:
@btime f();

  6.277 ms (200001 allocations: 21.36 MiB)


### How to handle it? -> More dots or more explicity

Great blog post by Steven G. Johnson: https://julialang.org/blog/2017/01/moredots ([related notebook](https://github.com/JuliaLang/www.julialang.org/blob/master/blog/_posts/moredots/More-Dots.ipynb))

In [48]:
function f()
    x = [1;5;6]
    for i in 1:100_000    
        for k in 1:3
            x[k] = x[k] + 2 * x[k]
        end
    end
    return x
end
@btime f();

  154.199 μs (1 allocation: 112 bytes)


In [49]:
M = rand(2,2)

2×2 Array{Float64,2}:
 0.503957  0.138657 
 0.314602  0.0567941

In [50]:
exp(M)

2×2 Array{Float64,2}:
 1.68664   0.186407
 0.422943  1.08549 

In [51]:
exp.(M)

2×2 Array{Float64,2}:
 1.65526  1.14873
 1.36971  1.05844

In [52]:
@which exp(M)

exp(A::Union{DenseArray{#s627,2}, Base.ReinterpretArray{#s627,2,S,A} where S where A<:Union{SubArray{T,N,A,I,true} where I<:Union{Tuple{Vararg{Real,N} where N}, Tuple{AbstractUnitRange,Vararg{Any,N} where N}} where A<:DenseArray where N where T, DenseArray}, Base.ReshapedArray{#s627,2,A,MI} where MI<:Tuple{Vararg{Base.MultiplicativeInverses.SignedMultiplicativeInverse{Int64},N} where N} where A<:Union{Base.ReinterpretArray{T,N,S,A} where S where A<:Union{SubArray{T,N,A,I,true} where I<:Union{Tuple{Vararg{Real,N} where N}, Tuple{AbstractUnitRange,Vararg{Any,N} where N}} where A<:DenseArray where N where T, DenseArray} where N where T, SubArray{T,N,A,I,true} where I<:Union{Tuple{Vararg{Real,N} where N}, Tuple{AbstractUnitRange,Vararg{Any,N} where N}} where A<:DenseArray where N where T, DenseArray}, SubArray{#s627,2,A,I,L} where L where I<:Tuple{Vararg{Union{Int64, AbstractRange{Int64}, Base.AbstractCartesianIndex},N} where N} where A<:Union{Base.ReinterpretArray{T,N,S,A} where S where A<:Union{SubArray{T,N,A,I,true} where I<:Union{Tuple{Vararg{Real,N} where N}, Tuple{AbstractUnitRange,Vararg{Any,N} where N}} where A<:DenseArray where N where T, DenseArray} where N where T, Base.ReshapedArray{T,N,A,MI} where MI<:Tuple{Vararg{Base.MultiplicativeInverses.SignedMultiplicativeInverse{Int64},N} where N} where A<:Union{Base.ReinterpretArray{T,N,S,A} where S where A<:Union{SubArray{T,N,A,I,true} where I<:Union{Tuple{Vararg{Real,N} where N}, Tuple{AbstractUnitRange,Vararg{Any,N} where N}} where A<:DenseArray where N where T, DenseArray} where N where T, SubArray{T,N,A,I,true} where I<:Union{Tuple{Vararg{Real,N} where N}, Tuple{AbstractUnitRange,Vararg{Any,N} where N}} where A<:DenseArray where N where T, DenseArray} where N where T, DenseArray}} where #s627<:Union{Complex{Float32}, Complex{Float64}, Float32, Float64}) in LinearAlgebra at D:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.3\LinearAlgebra\src\dense.jl:519

In [53]:
function f()
    x = [1;5;6]
    for i in 1:100_000
        x = x .+ 2 .* x
    end
    return x
end
@btime f();

  2.606 ms (100001 allocations: 10.68 MiB)


In [54]:
function f()
    x = [1;5;6]
    for i in 1:100_000
        x .= x .+ 2 .* x
    end
    return x
end
@btime f();

  290.599 μs (1 allocation: 112 bytes)


In [55]:
function f()
    x = [1;5;6]
    for i in 1:100_000
        @. x = x + 2*x
    end
    return x
end
@btime f();

  282.699 μs (1 allocation: 112 bytes)


### Extra performance: `@inbounds`

In [57]:
M[10,5]

BoundsError: BoundsError: attempt to access 2×2 Array{Float64,2} at index [10, 5]

In [58]:
function f()
    x = [1;5;6]
    @inbounds for i in 1:100_000    
        for k in 1:3
            x[k] = x[k] + 2*x[k]
        end
    end
    return x
end
@btime f();

  79.199 μs (1 allocation: 112 bytes)


# Gotcha 5: Abstract fields

In [59]:
using BenchmarkTools

In [60]:
struct MyType
    x::AbstractFloat
    y::AbstractString
end

f(a::MyType) = a.x^2 + sqrt(a.x)

f (generic function with 3 methods)

In [61]:
a = MyType(3.0, "test")

@btime f($a);

  49.293 ns (3 allocations: 48 bytes)


In [62]:
struct MyTypeConcrete
    x::Float64
    y::String
end

f(b::MyTypeConcrete) = b.x^2 + sqrt(b.x)

f (generic function with 4 methods)

In [63]:
b = MyTypeConcrete(3.0, "test")

@btime f($b);

  1.599 ns (0 allocations: 0 bytes)


Note that the latter implementation is **more than 30x faster**!

### How to handle it?

But what if I want to accept any kind of `AbstractFloat` and `AbstractString` in my type?

Use type parameters!

In [64]:
struct MyTypeParametric{A<:AbstractFloat, B<:AbstractString}
    x::A
    y::B
end

f(c::MyTypeParametric) = c.x^2 + sqrt(c.x)

f (generic function with 5 methods)

In [65]:
c = MyTypeParametric(3.0, "test")

MyTypeParametric{Float64,String}(3.0, "test")

From the type alone the compiler knows what the structure contains and can produce optimal code:

In [67]:
typeof(c)

MyTypeParametric{Float64,String}

In [66]:
@btime f($c);

  1.599 ns (0 allocations: 0 bytes)


In [68]:
c = MyTypeParametric(Float32(3.0), SubString("test"))

MyTypeParametric{Float32,SubString{String}}(3.0f0, "test")

In [69]:
typeof(c)

MyTypeParametric{Float32,SubString{String}}

In [70]:
@btime f($c);

  1.299 ns (0 allocations: 0 bytes)


In [72]:
Base.@kwdef struct MyStruct
    x
    y
end

MyStruct

In [73]:
MyStruct(y=1, x=2)

MyStruct(2, 1)

# Gotcha 6: Writing to global scope

In [74]:
# Try this in the Julia REPL
a = 0
for i in 1:10
    a += i
end

In [75]:
a

55

(For more information, see the "official" discussion here: https://github.com/JuliaLang/julia/issues/28789)

#### Take home message: again, just wrap things into functions.

# Gotcha 7: Column major order

In [76]:
M = rand(1000,1000);

function fcol(M)
    for col in 1:size(M, 2)
        for row in 1:size(M, 1)
            M[row, col] = 42
        end
    end
    nothing
end

function frow(M)
    for row in 1:size(M, 1)
        for col in 1:size(M, 2)
            M[row, col] = 42
        end
    end
    nothing
end

frow (generic function with 1 method)

In [77]:
@btime fcol($M)

  523.700 μs (0 allocations: 0 bytes)


In [78]:
@btime frow($M)

  1.536 ms (0 allocations: 0 bytes)


#### Take home message: fastest varying index goes first!

# Gotcha 8: Lazy operations

Let's say we want to calculate `X = M + (M' + 2*I)`.

In [79]:
using LinearAlgebra

In [81]:
M = [1 2; 3 4]
M + (M' + 2*I)

2×2 Array{Int64,2}:
 4   5
 5  10

Now let's assume that, for some reason, we want to implement it more explicitly. Something along the lines of

In [85]:
M'

2×2 Adjoint{Int64,Array{Int64,2}}:
 1  3
 2  4

In [87]:
copy(M')

2×2 Array{Int64,2}:
 1  3
 2  4

In [88]:
collect(M')

2×2 Array{Int64,2}:
 1  3
 2  4

In [91]:
collect(1:10)

10-element Array{Int64,1}:
  1
  2
  3
  4
  5
  6
  7
  8
  9
 10

In [82]:
function calc(M)
    X = M'
    X[1,1] += 2
    X[2,2] += 2
    M + X
end

calc (generic function with 1 method)

Let's check for correctness.

In [86]:
calc([1 2; 3 4]) == M + (M' + 2*I)

false

Somehow it's not correct!

### How to solve this?

The "issue" is that `M'` makes a lazy adjoint of `M`. It is just another way of looking at the same piece of memory. Hence, when we do `X[1,1] += 1` we are actually changing `M`, leading to a wrong result. We can heal this by enforcing a `copy`:

In [92]:
function calc_corrected(M)
    X = copy(M')
    X[diagind(X)] .+= 2
    M + X
end

calc_corrected (generic function with 1 method)

In [93]:
calc_corrected([1 2; 3 4]) == M + (M' + 2*I)

true

This isn't really an issue. In fact, this lazyness (+ allocation free identity matrix) is precisley the reason why the straightforward solution is fast!

In [94]:
function calc_straightforward(A)
    A + (A' + 2*I)
end

@btime calc($[1 2; 3 4]);
@btime calc_corrected($[1 2; 3 4]);
@btime calc_straightforward($[1 2; 3 4]);

  49.240 ns (2 allocations: 128 bytes)
  167.854 ns (5 allocations: 400 bytes)
  89.665 ns (3 allocations: 240 bytes)


### Extra tip: Comprehensions and generators

In [95]:
[1,2,3]

3-element Array{Int64,1}:
 1
 2
 3

In [96]:
[k for k in 1:10]

10-element Array{Int64,1}:
  1
  2
  3
  4
  5
  6
  7
  8
  9
 10

In [97]:
collect(k for k in 1:10)

10-element Array{Int64,1}:
  1
  2
  3
  4
  5
  6
  7
  8
  9
 10

The construct is known as a [comprehension](https://docs.julialang.org/en/v1/manual/arrays/#Comprehensions-1).

In [98]:
sum([k for k in 1:10])

55

To avoid the temporary array that the comprehension creates, we can also write the comprehension withouth square brackets. This creates a so-called [generator expression](https://docs.julialang.org/en/v1/manual/arrays/#Generator-Expressions-1).

In [99]:
sum(k for k in 1:10)

55

In [100]:
gen = (k for k in 1:10)

Base.Generator{UnitRange{Int64},var"#35#36"}(var"#35#36"(), 1:10)

In [101]:
collect(gen)

10-element Array{Int64,1}:
  1
  2
  3
  4
  5
  6
  7
  8
  9
 10

In [102]:
using BenchmarkTools

@btime sum([k for k in 1:10]);
@btime sum(k for k in 1:10);

  39.276 ns (1 allocation: 160 bytes)
  1.299 ns (0 allocations: 0 bytes)


# Core messages of this Notebook

* Gotcha 1: **Wrap code in self-contained functions** in performance critical applications, i.e. avoid global scope.
* Gotcha 2: Write **type-stable code** (check with `@code_warntype`).
* Gotcha 3: Use **views** instead of copies to avoid unnecessary allocations.
* Gotcha 4: Use **broadcasting (more dots)** to avoid temporary allocations in vectorized code (or write out loops).
* Gotcha 5: **Types should always have concrete fields.** If you don't know them in advance, use type parameters.


* Gotcha 6: Be aware of the **scoping rules** in non-Jupyter-notebook environments.
* Gotcha 7: Be aware of **column major order** when looping over arrays.
* Gotcha 8: Be aware of **lazy operations** like, for example, transpose.